### Carrega Lista de Processos, e baixa as decisões


In [ ]:
# Importando as Bibliotecas
import pandas as pd
import requests 
from bs4 import BeautifulSoup
import json
import pymongo
import yaml

In [ ]:
# Carrega Configurações do arquivo config
with open("config.yaml", "r") as ymlfile:
    cfg = yaml.full_load(ymlfile)

user = cfg["mongodb"]["user"]
passwd = cfg["mongodb"]["passwd"]
host = cfg["mongodb"]["host"]
dbmongo = cfg["mongodb"]["db"]


In [ ]:
# Cria Conexão com banco
url_conexao = "mongodb+srv://" + user + ":" + passwd + "@" + host + "/?retryWrites=true&w=majority"
client = pymongo.MongoClient(url_conexao)
db = client[dbmongo]

In [ ]:
# Coleções Mongo
col_processos = db['processo']
col_decisoes = db['decisoes']

In [ ]:
dados = pd.DataFrame(list(col_processos.find({'processo.cod_empresa':{'$in':['PORTOSEGURO','MAPFRE']}})))
df_processos = pd.json_normalize(dados['processo'])
df_processos.shape

In [ ]:
def pega_valores(soup,tag):
    valor = soup.select_one('strong:-soup-contains("'+ tag +'")')
    if valor: 
        valor = valor.parent.text.strip()
        valor = valor.replace(tag +':','').replace('\t','').replace('\n','').strip()
        return valor

In [ ]:
procs = df_processos['nro_processo']
procs
#procs = ['1008842-42.2021.8.26.0704','1003031-42.2022.8.26.0001','1015939-91.2018.8.26.0577','0008053-14.2021.8.26.0590']
#procs = list(df_procs['nro_processo'].head(100))
#df_procs2 = df_procs[1625:]
#procs = list(df_procs2['nro_processo'])

In [ ]:
dados = []

for proc in procs:
    url = "https://esaj.tjsp.jus.br/cjpg/pesquisar.do?conversationId=&dadosConsulta.pesquisaLivre=&tipoNumero=UNIFICADO&numeroDigitoAnoUnificado=1003031-42.2022&foroNumeroUnificado=0001&dadosConsulta.nuProcesso=" + proc
    
    try: 
        html = requests.get(url).content
    except: 
        pass
    soup = BeautifulSoup(html, 'html.parser')

    aviso = soup.select_one('div:-soup-contains("Não foi encontrado nenhum")')

    if aviso: 
        print("Processo: " + proc + " sem resultados")
        
    else:
        print("Processo: " + proc + " com resultados")
        classe = pega_valores(soup,'Classe')
        assunto = pega_valores(soup,'Assunto')
        magistrado = pega_valores(soup,'Magistrado')
        comarca = pega_valores(soup,'Comarca')
        foro = pega_valores(soup,'Foro')
        vara = pega_valores(soup,'Vara')
        data_disp = pega_valores(soup,'Data de Disponibilização')
        texto_decisao = soup.find('div',attrs={'align':'justify','style':'display: none;'}).text.strip()

        dados.append([proc,classe,assunto,magistrado,comarca,foro,vara,data_disp,texto_decisao])


In [ ]:
df_decisoes = pd.DataFrame(dados,columns=["nro_processo","classe","assunto","magistrado","comarca","foro","vara","data_disp","texto_decisao"])
df_decisoes['instancia'] = "1ª Instância"


In [ ]:
def carrega_decisao(linha):
    json_str = str(linha.to_json())
    json_str = '{"decisao":' + json_str + '}'
    col_decisoes.find_one_and_update({'decisao.nro_processo':linha['nro_processo']},{'$set':json.loads(json_str)},upsert=True)

In [ ]:
for index,decisao in df_decisoes.iterrows(): 
    carrega_decisao(df_decisoes.loc[index])
